In [83]:
import pandas as pd
import seaborn as sns

from Scripts.dirty_accuracy import injection
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Helper functions

Classifier functions

Decision Tree Classifier implementation
## LO TENIAMO COSI'?

In [84]:
def DTreeClassification(X_train, y_train, X_test, y_test):
    K = []
    training = []
    test = []
    scores = {}

    for k in range(1, 20):
        clf = DecisionTreeClassifier()
        clf.fit(X_train, y_train)

        training_score = clf.score(X_train, y_train)
        test_score = clf.score(X_test, y_test)
        K.append(k)

        training.append(training_score)
        test.append(test_score)
        scores[k] = [training_score, test_score]

    return scores

Other Classifier implementation

In [ ]:
def secondClassification(X_train, y_train, X_test, y_test):
    K = []
    training = []
    test = []
    scores = {}

    return scores

Print scores

In [85]:
def printScores(scores):
    for score in scores:
        print("Dataset dirty at: ", score['dirty'], "%")
        print(score['scores'][0],':', score['scores'][1])

Plot scores

In [86]:
def plotScores(scores, title):
    sns.set_palette("PuBuGn_d")
    for score in scores:
        labels = list(score['scores'].keys())
        labels.insert(0,0)
        scoresDF = pd.DataFrame.from_dict(score['scores'], orient='index', columns=['Train '+str(score['dirty']), 'Test '+str(score['dirty'])])
        plot = sns.lineplot(data=scoresDF)
        plot.set_xticks(range(len(score['scores'])+1))
        plot.set_xticklabels(labels)
        plot.set_ylim(0.2,1.1)
        plot.set(xlabel ="N Neighbors", ylabel = "Performance", title =title)
        plot.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)

First method detection and possibly correction

In [ ]:
def firstDetection(data):
    return data

Second method detection and possibly correction

In [ ]:
def secondDetection(data):
    return data

## Pipeline implementation


Read the dataset from CSV file

In [89]:
german = pd.read_csv("datasets/german.csv", sep=',')

Perform the injections of outliers and append the original clean dataset to the list

In [90]:
german_list = injection(df_pandas=german, seed=10, name='german', name_class='Risk')
german_list.append(german)

saved german-accuracy50%
saved german-accuracy60%
saved german-accuracy70%
saved german-accuracy80%
saved german-accuracy90%


Reverse the list on order to have
0 - Original Dataset
1 - 10% Dirty dataset
2 - 20% Dirty dataset
3 - 30% Dirty dataset
4 - 40% Dirty dataset
5 - 50% Dirty dataset

In [91]:
german_list.reverse()

### Functions to be repeated for each dataframe


Data conversion map to change all categorical columns into numerical values

In [ ]:
def DCmap(df):
    dataset = df.copy()
    for col in dataset:
        if dataset[col].dtype == object:
            to_replace = dataset[col].unique()
            value = []
            for i in range (len(to_replace)):
                value.append(i)
            dataset[col] = dataset[col].replace(to_replace=to_replace, value=value)
    return dataset

In [92]:
def computeScoresDT(dataframe_array):
    y = german['Risk']
    scores = []
    i = 0
    for df in dataframe_array:
        X = df
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
        sc = DTreeClassification(X_train, y_train, X_test, y_test)
        scores.append({'dirty': i, 'scores': sc})
        i = i + 10
    return scores

In [ ]:
def computeScoressecond(dataframe_array):
    y = german['risk']
    scores = []
    i = 0
    for df in dataframe_array:
        X = df.drop('class', axis=1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
        sc = secondClassification(X_train, y_train, X_test, y_test)
        scores.append({'dirty': i, 'scores': sc})
        i = i + 10
    return scores

### Performing Decision Tree Machine Learning classification
Dataset preparation

In [ ]:
dtscores = computeScoresDT(german_list)
printScores(dtscores)

In [ ]:
secondscore = computeScoressecond(german_list)
printScores(secondscore)

Find outliers and use replace them

Perform again the classification after outliers detection and print results

In [ ]:
printScores()

In [ ]:
printScores()

### Plot data preparation


In [ ]:
def toArray(score):
    t = []
    for s in score:
        t.append(s['scores'][1])
    return t
plotData = {
    'dirty': [0, 10, 20, 30, 40, 50],
    'knc_pre': toArray(knscores),
    'gc_pre': toArray(gcscore),
    'knc_zs_post': toArray(cleaned_knscores),
    'gc_zs_post': toArray(cleaned_gcscores)
}
plotDF = pd.DataFrame(plotData)
sns.set()
fig, axes = plt.subplots(4, 1, sharex=False, figsize=(7,14))

sns.lineplot(ax=axes[0], x='dirty', y='knc_pre', data=plotDF)
sns.lineplot(ax=axes[0], x='dirty', y='knc_zs_post', data=plotDF)
axes[0].legend(['Pre detection', 'Post detection'])
axes[0].set(xlabel='Dirty percentage', ylabel='Performance')
axes[0].set_ylim(0.2,1.1)
axes[0].set_title('KNN + ZS')

sns.lineplot(ax=axes[1], x='dirty', y='gc_pre', data=plotDF)
sns.lineplot(ax=axes[1], x='dirty', y='gc_zs_post', data=plotDF)
axes[1].legend(['Pre detection', 'Post detection'])
axes[1].set(xlabel='Dirty percentage', ylabel='Performance')
axes[1].set_ylim(0.2,1.1)
axes[1].set_title('GC + ZS')

axes[2].legend(['Pre detection', 'Post detection'])
axes[2].set(xlabel='Dirty percentage', ylabel='Performance')
axes[2].set_ylim(0.2,1.1)
axes[2].set_title('KNN + Unknown')

axes[3].legend(['Pre detection', 'Post detection'])
axes[3].set(xlabel='Dirty percentage', ylabel='Performance')
axes[3].set_ylim(0.2,1.1)
axes[3].set_title('GC + Unknown')
fig.tight_layout()

In [ ]:
sns.set()
fig, axes = plt.subplots(3, 1, sharex=False, figsize=(7,18))

sns.lineplot(ax=axes[0], x='dirty', y='knc_pre', data=plotDF)
sns.lineplot(ax=axes[0], x='dirty', y='gc_pre', data=plotDF)
axes[0].legend(['Pre detection KNC', 'Pre detection GC'])
axes[0].set(xlabel='Dirty percentage', ylabel='Performance')
axes[0].set_ylim(0.2,1.1)
axes[0].set_title('Pre detection performances')

sns.lineplot(ax=axes[1], x='dirty', y='knc_zs_post', data=plotDF)
sns.lineplot(ax=axes[1], x='dirty', y='gc_zs_post', data=plotDF)
axes[1].legend(['Post detection KNC', 'Post detection GC'])
axes[1].set(xlabel='Dirty percentage', ylabel='Performance')
axes[1].set_ylim(0.2,1.1)
axes[1].set_title('Post detection (ZS) performances')

#sns.lineplot(ax=axes[2], x='dirty', y='', data=plotDF)
#sns.lineplot(ax=axes[2], x='dirty', y='', data=plotDF)
axes[2].legend(['Post detection KNC', 'Post detection GC'])
axes[2].set(xlabel='Dirty percentage', ylabel='Performance')
axes[2].set_ylim(0.2,1.1)
axes[2].set_title('Post detection (Unknown) performances')